In [1]:
import numpy as np
from scipy.stats import norm
from scipy import integrate
import torch

from bo_functions import ei_approx_ln_term
from bo_functions import calc_ei_emulator

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
error_best = 1.0376255530659095
pred_mean = np.array([-8.14817608, -1.95393033,  0.00769104701, -2.03767010, -2.11993305])
pred_stdev = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])
pred_var = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])**2
y_target = np.array([-14.0031178, -2.99270996,  0.00217820788,  0.991009082, 5.97513219])
explore_bias = torch.tensor([1])

In [3]:
error_best = 1.0376255530659095
pred_mean = np.array([-8.14817608, -1.95393033,  0.00769104701, -2.03767010, -2.11993305])
pred_stdev = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])
pred_var = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])**2
y_target = np.array([-14.0031178, -2.99270996,  0.00217820788,  0.991009082, 5.97513219])
explore_bias = torch.tensor([1])

In [ ]:
def ei_approx_ln_term(epsilon, error_best, pred_mean, pred_stdev, y_target, ep): 
    """ 
    Calculates the integrand of expected improvement of the 3 input parameter GP using the log version
    Parameters
    ----------
        epsilon: The random variable. This is the variable that is integrated w.r.t
        error_best: float, the best predicted error encountered
        pred_mean: ndarray, model mean
        pred_stdev: ndarray, model stdev
        y_target: ndarray, the expected value of the function from data or other source
        ep: float, the numerical bias towards exploration, zero is the default
    
    Returns
    -------
        ei: ndarray, the expected improvement for one term of the GP model
    """
#     EI = ( (error_best - ep) - np.log( (y_target - pred_mean - pred_stdev*epsilon)**2 ) )*norm.pdf(epsilon)

    ei_term_2_integral = np.log( abs((y_target - pred_mean - pred_stdev*epsilon)) )*norm.pdf(epsilon)
#     ei_term_2_integral = np.log( (y_target - pred_mean - pred_stdev*epsilon)**2 )*norm.pdf(epsilon)
    return ei_term_2_integral
    
def calc_ei_emulator(error_best,pred_mean,pred_var,y_target, explore_bias=0.0, obj = "obj"): #Will need obj toggle soon
    """ 
    Calculates the expected improvement of the 3 input parameter GP
    Parameters
    ----------
        error_best: float, the best predicted error encountered
        pred_mean: ndarray, model mean
        pred_var: ndarray, model variance
        y_target: ndarray, the expected value of the function from data or other source
        explore_bias: float, the numerical bias towards exploration, zero is the default
        obj: str, LN_obj or obj, determines whether log or regular EI function is calculated
    
    Returns
    -------
        ei: ndarray, the expected improvement for one term of the GP model
    """
    #Asserts that f_pred is a float, and y_target is an ndarray
    assert isinstance(error_best, (float,int))==True, "error_best must be a float or integer"
    
    #Coverts any tensors given as inputs to ndarrays         
    #Checks for equal lengths
    assert isinstance(y_target, float)==True, "y_target, pred_mean, and pred_var must be floats"
    assert isinstance(pred_mean, float)==True, "y_target, pred_mean, and pred_var must be floats"
    assert isinstance(pred_var, float)==True, "y_target, pred_mean, and pred_var must be floats"
    
    #Defines standard devaition
    pred_stdev = np.sqrt(pred_var) #1xn
    
    #If variance is zero this is important
    if obj == "obj":
        with np.errstate(divide = 'warn'):
            #Creates upper and lower bounds and described by Alex Dowling's Derivation
#             bound_a = ((y_target - pred_mean) +np.sqrt(error_best - explore_bias))/pred_stdev #1xn
#             bound_b = ((y_target - pred_mean) -np.sqrt(error_best - explore_bias))/pred_stdev #1xn
            bound_a = ((y_target - pred_mean) +np.sqrt(error_best*explore_bias))/pred_stdev #1xn
            bound_b = ((y_target - pred_mean) -np.sqrt(error_best*explore_bias))/pred_stdev #1xn
            bound_lower = np.min([bound_a,bound_b])
            bound_upper = np.max([bound_a,bound_b])        

            #Creates EI terms in terms of Alex Dowling's Derivation
            ei_term1_comp1 = norm.cdf(bound_upper) - norm.cdf(bound_lower) #1xn
#             ei_term1_comp2 = (error_best - explore_bias) - (y_target - pred_mean)**2 #1xn
            ei_term1_comp2 = (error_best*explore_bias) - (y_target - pred_mean)**2 #1xn

            ei_term2_comp1 = 2*(y_target - pred_mean)*pred_stdev #1xn
            ei_eta_upper = -np.exp(-bound_upper**2/2)/np.sqrt(2*np.pi)
            ei_eta_lower = -np.exp(-bound_lower**2/2)/np.sqrt(2*np.pi)
            ei_term2_comp2 = (ei_eta_upper-ei_eta_lower)

            ei_term3_comp1 = bound_upper*ei_eta_upper #1xn
            ei_term3_comp2 = bound_lower*ei_eta_lower #1xn

            ei_term3_comp3 = (1/2)*math.erf(bound_upper/np.sqrt(2)) #1xn
            ei_term3_comp4 = (1/2)*math.erf(bound_lower/np.sqrt(2)) #1xn     

            ei_term3_psi_upper = ei_term3_comp1 + ei_term3_comp3 #1xn
            ei_term3_psi_lower = ei_term3_comp2 + ei_term3_comp4 #1xn
            ei_term1 = ei_term1_comp1*ei_term1_comp2 #1xn

            ei_term2 = ei_term2_comp1*ei_term2_comp2 #1xn
            ei_term3 = -pred_var*(ei_term3_psi_upper-ei_term3_psi_lower) #1xn
            EI = ei_term1 + ei_term2 + ei_term3 #1xn
    else:
#         print("It's working")
        with np.errstate(divide = 'warn'):
            #Creates upper and lower bounds and described by Alex Dowling's Derivation
#             bound_a = ((y_target - pred_mean) +np.sqrt(np.exp(error_best - explore_bias)))/pred_stdev #1xn
#             bound_b = ((y_target - pred_mean) -np.sqrt(np.exp(error_best - explore_bias)))/pred_stdev #1xn
            bound_a = ((y_target - pred_mean) +np.sqrt(np.exp(error_best*explore_bias)))/pred_stdev #1xn
            bound_b = ((y_target - pred_mean) -np.sqrt(np.exp(error_best*explore_bias)))/pred_stdev #1xn
            bound_lower = np.min([bound_a,bound_b])
            bound_upper = np.max([bound_a,bound_b])
            
            args = (error_best, pred_mean, pred_stdev, y_target, explore_bias)
#             print(bound_lower,bound_upper)
#             print(error_best, pred_mean, pred_stdev, y_target, explore_bias)
            #This first way is very slow
#             ei, abs_err = integrate.quad(ei_approx_ln_term, bound_lower, bound_upper, args = args) 
            #This 2nd way throws the error -> too many values to unpack (expected 3) even though 3 values are being unpacked unless you do it like this and not, EI, abs_err, infordict =
            ei_term_1 = (error_best*explore_bias)*( norm.cdf(bound_upper)-norm.cdf(bound_lower) )
            ei_term_2_out = integrate.quad(ei_approx_ln_term, bound_lower, bound_upper, args = args, full_output = 1)
            ei_term_2 = (-2)*ei_term_2_out[0] 
#             ei_term_2 = (-1)*ei_term_2_out[0] 
            term_2_abs_err = ei_term_2_out[1]
            EI = ei_term_1 + ei_term_2
#             print(EI)
   
    ei = EI         
    return ei


In [4]:
%%time
ei = 0
#Try bringing out the 2 and taking the abs()
for i in range(len(y_target)):
    with np.errstate(divide = 'warn'):
        bound_a = ((y_target[i] - pred_mean[i]) + np.sqrt(np.exp(error_best*explore_bias)))/pred_stdev[i] #1xn
        bound_b = ((y_target[i] - pred_mean[i]) - np.sqrt(np.exp(error_best*explore_bias)))/pred_stdev[i] #1xn
        bound_lower = np.min([bound_a,bound_b])
        bound_upper = np.max([bound_a,bound_b])
#         print(bound_lower,bound_upper)

        args = (error_best, pred_mean[i], pred_stdev[i], y_target[i], explore_bias)

        ei_term_1 = (error_best*explore_bias)*( norm.cdf(bound_upper)-norm.cdf(bound_lower) )
    #         print(ei_term_1)
        ei_term_2_out = integrate.quad(ei_approx_ln_term, bound_lower, bound_upper, args = args, full_output = 1)
        ei_term_2 = (-1)*ei_term_2_out[0] 
#         print(ei_term_1, ei_term_2)
        term_2_abs_err = ei_term_2_out[1]
        EI = ei_term_1 + ei_term_2
        ei += EI

print(ei)

#Note: 1min + 45sec per iteration

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creatin

tensor([2.4493], dtype=torch.float64)
CPU times: user 303 ms, sys: 9.01 ms, total: 312 ms
Wall time: 302 ms


In [5]:
%%time
ei = 0

for i in range(len(y_target)):
    ei_x = calc_ei_emulator(error_best,pred_mean[i],pred_var[i],y_target[i], explore_bias, obj = "LN_obj")
    ei += ei_x
    
print(ei)

tensor([3.2931], dtype=torch.float64)
CPU times: user 296 ms, sys: 1.97 ms, total: 298 ms
Wall time: 295 ms


In [6]:
%%time
ei = 0

for i in range(len(y_target)):
    ei_x = calc_ei_emulator(error_best,pred_mean[i],pred_var[i],y_target[i], explore_bias, obj = "LN_obj")
    ei += ei_x
    
print(ei)

tensor([3.2931], dtype=torch.float64)
CPU times: user 295 ms, sys: 8.52 ms, total: 303 ms
Wall time: 296 ms
